<a href="https://colab.research.google.com/github/hbb2019/nanoLM/blob/main/science_label_ipynb%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
file_path = '/content/drive/My Drive/BAAI/industry_instruction_science_research_train.jsonl'

In [4]:
df = pd.read_json(file_path, lines=True)

In [5]:
df = df[:10000] # 时间原因，分段计算

In [6]:
print(f'行数:{df.shape[0]}')
print(f'列数:{df.shape[1]}')

行数:10000
列数:7


In [7]:
df.iloc[1]

,1
deita_score,12.915536
rw_score,-5.125
id,generate_from_given_text
instruction,
lang,en
length,38
conversations,"[{'from': 'human', 'value': 'How does the clus..."


In [8]:
# list
df.iloc[1]['conversations']

[{'from': 'human',
  'value': 'How does the clustering of microearthquakes help seismologists understand what initiates and governs an earthquake, and what insights do researchers like Ross Stein hope to gain from this?'},
 {'from': 'gpt',
  'value': "The clustering of microearthquakes helps seismologists understand what initiates and governs an earthquake by providing insights into the stress transfer and fault interaction processes. By analyzing the locations and patterns of microearthquakes, researchers like Ross Stein hope to gain a better understanding of how earthquakes are triggered and how stress is transferred along faults. This knowledge can help improve earthquake forecasting and hazard assessment. Stein's research focuses on the interaction between earthquakes and faults, and he hopes to learn more about the underlying mechanisms that control earthquake behavior."}]

In [9]:
# 提取所有行的对话字符串
def extract_conversation_strings(df):
    conversation_strings = []
    for _, row in df.iterrows():
        try:
            conversation_list = row['conversations']
            if not isinstance(conversation_list, list) or len(conversation_list) != 2:
                conversation_strings.append("unknown")
                continue
            human_value = conversation_list[0].get('value', '')
            gpt_value = conversation_list[1].get('value', '')
            QA_part = human_value + " " + gpt_value
            conversation_strings.append(QA_part)
        except Exception as e:
            print(f"错误处理行: {e}")
            conversation_strings.append("error")
    return conversation_strings

# 假设 df 是您的 DataFrame
# 提取所有对话字符串成列表
conversation_list = extract_conversation_strings(df)

In [10]:
conversation_list[:3]

['在5G网络开发过程中，有哪些常见的安全挑战需要解决？ 在5G网络开发过程中，有几个常见的安全挑战需要解决。首先是网络架构的安全性，包括如何保护核心网络、边缘网络和终端设备的安全。其次是用户数据的隐私保护，如何确保用户的个人信息和通信内容在传输和存储过程中的安全性。另外，物联网设备的安全性也是一个重要的挑战，因为5G网络将连接大量的物联网设备，如何保护这些设备免受攻击和滥用是一个关键问题。此外，虚拟化和云计算技术的应用也带来了新的安全挑战，如如何保护虚拟化环境的安全和云服务提供商的安全。总的来说，在5G网络开发过程中，需要解决这些常见的安全挑战，以确保网络的安全和可靠性。',
 "How does the clustering of microearthquakes help seismologists understand what initiates and governs an earthquake, and what insights do researchers like Ross Stein hope to gain from this? The clustering of microearthquakes helps seismologists understand what initiates and governs an earthquake by providing insights into the stress transfer and fault interaction processes. By analyzing the locations and patterns of microearthquakes, researchers like Ross Stein hope to gain a better understanding of how earthquakes are triggered and how stress is transferred along faults. This knowledge can help improve earthquake forecasting and hazard assessment. Stein's research focuses on the interaction betwe

In [11]:
QA_labels = [
    "math",
    "chemistry",
    "physics",
    "biology",
    "computer_science",
    "earth_science",
    "astronomy",
    "environmental_science",
    "material_science",
    "engineering",
    "medicine",
    "statistics",
    "psychology",
    "neuroscience",
    "data_science"
]

In [12]:
# from google.colab import userdata
# hf_token = userdata.get('HF_TOKEN')
# from huggingface_hub import login
# login()  # 自动使用 HF_TOKEN

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [14]:
from transformers import pipeline
# 加载模型
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=0)
print(classifier.model.device)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


cuda:0


In [15]:
from tqdm import tqdm

In [16]:
# 要分类的文本
sequence = conversation_list
batch_size = 10
num_batches = len(sequence) // batch_size + (1 if len(sequence) % batch_size != 0 else 0)


In [ ]:
progress_bar = tqdm(total=len(sequence), desc="正在处理文本", unit="条")
# 初始化结果列表
all_results = []
# 分批处理
for i in range(0, len(sequence), batch_size):
    # 获取当前批次
    batch = sequence[i:i + batch_size]
    # 对当前批次运行分类器
    batch_results = classifier(batch, QA_labels)
    # 将结果添加到总结果列表
    all_results.extend(batch_results)
    # 更新进度条，步长为当前批次的大小
    progress_bar.update(len(batch))

# 关闭进度条
progress_bar.close()

正在处理文本:  81%|████████  | 8110/10000 [1:35:43<20:22,  1.55条/s]

In [ ]:
subject_labels = [v['labels'][0] for v in all_results]

In [ ]:
df['subject_label'] = subject_labels

In [ ]:
print(df[['conversations', 'subject_label']].head(3))

In [ ]:
from datetime import datetime
import os

# 构造新文件名（添加日期）
# 获取当前日期（2025-04-22）
current_date = datetime(2025, 4, 23).strftime('%Y-%m-%d')  # 格式化为 '2025-04-22'

# 提取文件夹路径和原始文件名
folder_path = os.path.dirname(file_path)  # '/content/drive/My Drive/BAAI'
file_name = os.path.basename(file_path)   # 'train-00000-of-00015.parquet'

# 分割文件名和扩展名
file_name_base, file_extension = os.path.splitext(file_name)  # 'train-00000-of-00015', '.parquet'

# 构造新文件名
new_file_name = f"{file_name_base}_{current_date}{file_extension}"  # 'train-00000-of-00015_2025-04-22.parquet'

# 构造新文件的完整路径
new_file_path = os.path.join(folder_path, new_file_name)  # '/content/drive/My Drive/BAAI/train-00000-of-00015_2025-04-22.parquet'

# 步骤 3：保存 DataFrame 到新文件
df.to_json(new_file_path, orient='records', lines=True, index=False, force_ascii=False)

# 打印确认
print(f"文件已保存到: {new_file_path}")